In [14]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras import *
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.optimizers import Adam
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D, GlobalAveragePooling2D
#from keras.layers.advanced_activations import LeakyReLU
from keras.preprocessing.image import ImageDataGenerator
#from sklearn.model_selection import train_test_split
from skimage import io
import os
import scipy.misc
from scipy.misc import imread, imresize
from keras import regularizers
import csv

Function to get training and validation images.

In [15]:
import cv2

def load_images(path):
    img_data = []
    labels = []
    idx_to_label = []
    i = -1
    for fruit in os.listdir(path):
        fruit_path = os.path.join(path,fruit)
        labels.append(fruit)
        i = i+1
        for img in os.listdir(fruit_path):
            img_path = os.path.join(fruit_path,img)
            image = cv2.imread(img_path)
            image = cv2.resize(image, (64, 64))
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            img_data.append(image)
            idx_to_label.append(i)
    return np.array(img_data),np.array(idx_to_label),labels


Calling the function.

In [16]:
trn_data_path = 'C:/Users/Pushkar/Downloads/Datasets/fruits-360/Training'
val_data_path = 'C:/Users/Pushkar/Downloads/Datasets/fruits-360/Validation'
X_train,y_train,label_data = load_images(trn_data_path)
print(X_train.shape)
print(y_train.shape)
print(label_data)
X_test,y_test,label_data_garbage = load_images(val_data_path)

(32426, 64, 64, 3)
(32426,)
['Apple Braeburn', 'Apple Golden 1', 'Apple Golden 2', 'Apple Golden 3', 'Apple Granny Smith', 'Apple Red 1', 'Apple Red 2', 'Apple Red 3', 'Apple Red Delicious', 'Apple Red Yellow', 'Apricot', 'Avocado', 'Avocado ripe', 'Banana', 'Banana Red', 'Cactus fruit', 'Cantaloupe 1', 'Cantaloupe 2', 'Carambula', 'Cherry 1', 'Cherry 2', 'Cherry Rainier', 'Clementine', 'Cocos', 'Dates', 'Granadilla', 'Grape Pink', 'Grape White', 'Grape White 2', 'Grapefruit Pink', 'Grapefruit White', 'Guava', 'Huckleberry', 'Kaki', 'Kiwi', 'Kumquats', 'Lemon', 'Lemon Meyer', 'Limes', 'Litchi', 'Mandarine', 'Mango', 'Maracuja', 'Melon Piel de Sapo', 'Nectarine', 'Orange', 'Papaya', 'Passion Fruit', 'Peach', 'Peach Flat', 'Pear', 'Pear Abate', 'Pear Monster', 'Pear Williams', 'Pepino', 'Pineapple', 'Pitahaya Red', 'Plum', 'Pomegranate', 'Quince', 'Raspberry', 'Salak', 'Strawberry', 'Tamarillo', 'Tangelo']


One hot encoding.

In [17]:
num_of_classes = 65
Y_train = np_utils.to_categorical(y_train, num_of_classes)
Y_test = np_utils.to_categorical(y_test, num_of_classes)


Model.
I played around with the dropout values and achieved better score at 0.7
activations can be Relu, elu, LeakyRelu.

In [18]:
model = Sequential()

model.add(Conv2D(16, (3, 3), input_shape=(64,64,3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(16, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(32,(3,3 )))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Flatten())

# Fully connected layer
model.add(Dense(512))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.7))
model.add(Dense(65))

model.add(Activation('softmax'))

In [19]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 62, 62, 16)        448       
_________________________________________________________________
batch_normalization_7 (Batch (None, 62, 62, 16)        64        
_________________________________________________________________
activation_8 (Activation)    (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 60, 60, 16)        2320      
_________________________________________________________________
batch_normalization_8 (Batch (None, 60, 60, 16)        64        
_________________________________________________________________
activation_9 (Activation)    (None, 60, 60, 16)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 16)        0         
__________

In [20]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(),metrics=['accuracy'])

No augmentation.

In [24]:
model.fit(X_train,Y_train, epochs=1,batch_size = 32,
                    validation_data=(X_test,Y_test))

Train on 32426 samples, validate on 10903 samples
Epoch 1/1
32426/32426 [==============================] - 29s 879us/step - loss: 0.0816 - acc: 0.9735 - val_loss: 0.1690 - val_acc: 0.9431


In [25]:
model.fit(X_train,Y_train, epochs=1,batch_size = 32,
                    validation_data=(X_test,Y_test))

Train on 32426 samples, validate on 10903 samples
Epoch 1/1
32426/32426 [==============================] - 28s 878us/step - loss: 0.0561 - acc: 0.9812 - val_loss: 0.1440 - val_acc: 0.9487


In [26]:
score = model.evaluate(X_test, Y_test)
print()
print('Test loss: ', score[0])
print('Test Accuracy', score[1])



10903/10903 [==============================] - 4s 327us/step

Test loss:  0.14403905968
Test Accuracy 0.94872970742


In [27]:
predictions = model.predict_classes(X_test)

predictions = list(predictions)
actuals = list(y_test)

sub = pd.DataFrame({'Actual': actuals, 'Predictions': predictions})
sub.to_csv('./output_cnn.csv', index=False)


Augmentation.

In [28]:
gen = ImageDataGenerator(width_shift_range=.2, 
                             height_shift_range=.2,
                         zoom_range = 0.1,
                        horizontal_flip = 'True')

test_gen = ImageDataGenerator()
train_generator = gen.flow(X_train, Y_train, batch_size=32)
test_generator = test_gen.flow(X_test, Y_test, batch_size=32)

In [29]:
model.fit_generator(train_generator, epochs=5, 
                    validation_data=test_generator)

Epoch 1/5
1014/1014 [==============================] - 42s 42ms/step - loss: 0.8392 - acc: 0.7528 - val_loss: 0.3080 - val_acc: 0.8921
Epoch 2/5
1014/1014 [==============================] - 40s 40ms/step - loss: 0.3594 - acc: 0.8765 - val_loss: 0.3118 - val_acc: 0.8920
Epoch 3/5
1014/1014 [==============================] - 41s 41ms/step - loss: 0.2503 - acc: 0.9111 - val_loss: 0.3371 - val_acc: 0.8976
Epoch 4/5
1014/1014 [==============================] - 41s 40ms/step - loss: 0.2237 - acc: 0.9218 - val_loss: 0.0713 - val_acc: 0.9683
Epoch 5/5
1014/1014 [==============================] - 42s 41ms/step - loss: 0.1994 - acc: 0.9301 - val_loss: 0.1077 - val_acc: 0.9586
